# Event-related averages of fMRI data and fMRI impulse response function


We will introduce the way we formalize the organization of an experiment in a *design matrix*.

# Goals

* Experimental Design (continued): Block Design vs. Event-Related Design
* Block Design Analysis (Average Response)
    * Compute averages of activity for different types of events in a localizer experiment.
* Understand how responses to stimuli evolve over time
* Accounting for the fMRI hemodynamic response function (HRF)


<a id="Experimental_Design"></a>
# Experimental design

Cognitive neuroscientists conduct fMRI experiments to learn something about how cognition is related to brain responses, and where in the brain those responses occur. Cognition is a term that describes mental processes such as attention, memory, perception, decision making, planning, language, motor control or emotion. Since we can't measure mental phenomena directly, we either characterize properties of stimuli used or record behavior as a proxy for the cognitive processes that we believe underlie those behaviors. Before we learn the techniques that allow neuroscientists to relate those stimuli and behaviors to brain responses, we'll first spend some time discussing how fMRI experiments are designed, and the reasons behind some of those design decisions. 

Broadly speaking, we must be concerned with 2 aspects of an experiment:


1. **Cognitive Process Elicitation:** The experiment must be designed to elicit the cognitive process or processes that we're trying to understand. This is generally done by presenting the participant with a stimulus, asking them to complete some task, or a combination of both. For example:
    * A stimulus-only experiment could involve simply showing the participant some movies while they lie in the MRI scanner and look wherever they want at the movie. This would elicit neural activity in the visual system, and the scientist could study which parts of the visual system respond to which types of stimuli.
    * A task-only experiment could involve asking the subject to start counting from 0 to 100 saying one number every second. This would elicit neural activity in the speech production region of the brain.
    * A stimulus and task experiment could involve showing participants a set of images, showing them a second set of images and asking them to indicate if they had already seen this image. This would elicit neural activity in regions responsible for encoding and retrieving memories.
    
2. **Technical Considerations:** Any experiment involves taking measurements of the phenomena being studied. Considering all the ways your measurement tools affect the data collection is crucial to collecting data that is high quality. While there are many considerations when using an MRI scanner to collect BOLD data, we will be primarily concerned with accounting for two properties of the BOLD signal: 
    * The BOLD signal is **slow**
    * The BOLD signal is **noisy**
  
## Block vs. event-related designs

While the first concern just mentioned (eliciting cognitive processes to study) differs from experiment to experiment, the technical considerations of doing fMRI studies are common to all. Let's explore 2 broad classes of experimental designs that deal with the two technical considerations mentioned above:
<a id="Block_Design"></a>

1\. **Block Design:** Block designs consist of presenting many stimuli (or trials types) of the same type together in a row, or a block. The BOLD response to this block of stimuli can then be averaged to investigate which brain regions respond more to a particular stimuli than others. This averaging of TRs in a block design helps to improve the SNR. This improvement in SNR from using a block design comes at a cost, however: block designs are not temporally efficient, they take a long time to show just a few different types of stimuli. Since time in the MRI machine is expensive (several hundred dollars an hour), this is not an efficient way to conduct fMRI research. Here's a schematic of an example block design alternating blocks of doing a task (T) or rest (C), along with a BOLD signal that just responds to the task.

![alt text](../../data/images/block_bold.png "block_bold.png")

<a id="Event_Related_Design"></a>
2\. **Event-Related Design:** In a fast event-related design experiment, many different stimuli are presented at much shorter intervals. While this design is more temporally efficient (which allows for more complex experiments), a potential problem with event-related designs is that the BOLD signal responds slowly (peaks at about 5 seconds), and so if we show many different types of stimuli in a row, the BOLD signal response from one stimulus will "bleed over" into the TRs of the neighboring stimuli. Thus, if we want to mathematically describe how the brain responds to a given stimulus, we have to incorporate this delay in order to be accurate. See the image below for an illustration of how this might be done.

![alt text](../../data/images/event_related.PNG "event_related.PNG")

This problem of "bleeding over" has a solution however, and it involves characterizing the Hemodynamic Response Function (HRF) as we'll see later in this lecture. Since event-related designs are more complex. We'll see later on today how accounting for the HRF can even benefit block design experiments!

In [ ]:
# Load necessary libraries
import matplotlib.pyplot as plt
import cortex as cx
import neurods as nds
import numpy as np
from scipy.stats import zscore
import nibabel
import os

In [ ]:
# Matplotlib defaults
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib widget

# Load data

Same as previously: load using nibabel, use get_data() method of the nibabel data object, transpose resulting data array, and zscore the data. For now, we will treat this as a generic data set (next week, we will learn more about the experiment that generated this data as we begin to actually analyze the data).

In [ ]:
sub, xfm = 's01', 'catloc'
basedir = '../../data/fMRI/categories/'
fname = os.path.join(basedir, 's01_categories_01.nii.gz')
mask = cx.db.get_mask(sub, xfm, type='cortical')

nii = nibabel.load(fname)
data = nii.get_fdata().T.astype(np.float64)
data = data[:, mask]
data = zscore(data, axis=0)

print("Data dimensions are: ", data.shape)

In [ ]:
# Unmask
data3d = nds.fmri.unmask(data[0], mask, bg_value=np.nan)
data4d = nds.fmri.unmask(data, mask, bg_value=0)
print("data3d dimensions are: ", data3d.shape)
print("data4d dimensions are: ", data4d.shape)

# let's plot it
def slice_3d_array(vol, axis=0, **kwargs):
    
    """Function to plot slices of a 3D array along a given axis."""
    
    nslices = vol.shape[axis]
    fig = plt.figure(figsize=(10,10))
    subplot_size = int(np.ceil(np.sqrt(nslices)))
    print("{} slices will be visualized".format(nslices))

    for s in range(nslices):
        ax = fig.add_subplot(subplot_size, subplot_size, s+1)
        if axis==0:
            slices = vol[s,:,:]
        elif axis==1:
            slices = vol[:,s,:]
        elif axis==2:
            slices = vol[:,:,s]

        plt.imshow(slices, **kwargs)
        ax.axis('off')
        
slice_3d_array(data3d, axis=0, cmap=plt.cm.RdBu_r, vmin=-3, vmax=3)

## Breakout session
> Two weeks ago, we made a plot of a single slice over time. Can you make a plot like that using slice_3d_array? (Plot the 8th slice over time). 

In [ ]:
### STUDENT ANSWER


# Quick pycortex review

In [ ]:
# Create a volume
vkw = dict(mask=mask, cmap='RdBu_r', vmin=-3, vmax=3)
vol = cx.Volume(data[0], sub, xfm, **vkw)

In [ ]:
# Show a flamtap
# (If you get an XMLSyntaxError, ignore them; they are not important for now.)
try:
    _ = cx.quickflat.make_figure(vol)
except:
    pass

## Load description of experiment 
The experiment we have been working with is a *localizer* experiment. It is designed to find areas of the brain that respond to particular visual categories of objects: faces, bodies, and places. It also reveals areas that respond more to objects than to scrambled versions of the same objects. This experiment is a simple replication of past work, and is commonly done as a first step to locate (or localize) a region of interest for further analysis in a subsequent experiment.

For the localizer experiment, images from each category were presented in a block design. This means that images from the one category were shown one after another for a "block" of 20 seconds (10 TRs), followed by images from another category for a block of 20 seconds, and so on.

![alt text](../../data/images/CategoryLocalizerDesign.001.png "CategoryLocalizerDesign.001.png")


To analyze the data from this experiment at all, we need to know when the blocks for each category (faces, bodies, places, objects, and scrambled objects) began and ended. This information is stored in a *design matrix*, which we load below.

In [ ]:
design = np.load(os.path.join(basedir, 'experiment_design.npz'))
print('Experiment design variables: ', sorted(design.keys()))

In [ ]:
conditions = design['conditions'].tolist()
print('Conditions: ', conditions)
design_run1 = design['run1']
print('Design shape: ', design_run1.shape)

It's often useful to show a design matrix as an image:

In [ ]:
plt.figure()
_ = plt.imshow(design_run1.T)
plt.show()

## Breakout session
> What are the dimensions here? Label the axes on the figure above!

In [ ]:
### STUDENT ANSWER


# Find condition onsets
Last week for homework you wrote a function to compute event-related averages of data, given condition onset times. Here's a good version of such a function:

In [ ]:
nds.fmri.compute_event_avg??

Here, we don't have explicit times, only logical indices for which timepoints belong to which conditions - so we need to compute when the condition onsets were to use our function, or we need to write a new function! Let's stick with the old one, as we'll use it later, and just find the condition onsets.

> Find the onsets for each condition! And make your code into a re-usable function to find onsets

In [ ]:
# Find condition onsets
cond = 0
onsets, = np.nonzero(design_run1[:, cond])
onsets = [o for o in onsets if o-1 not in onsets]
print(onsets)
plt.figure()
# Sanity check: did we do that right? 
plt.plot(design_run1[:,0])
# or
#plt.stem(design_run1[:,0])
plt.plot(onsets, [1, 1], 'ro')
plt.ylim([-.5, 1.5])
plt.show()

# Code
def get_onsets(design, cond):
    """Convert condition design matrix of 1s and 0s to condition onsets 
    for a specific condition"""
    # Note fancy syntax to avoid tuple output
    onsets, = np.nonzero(design[:, cond])
    # Doesn't have to be an array, but why not
    onsets = np.array([o for o in onsets if o-1 not in onsets])
    return onsets

SO: Compute event averages for one condition!

In [ ]:
cond = 'faces'
idx = conditions.index(cond)
cond_onsets = get_onsets(design_run1, idx)
n_time_points = 10
cond_avg = nds.fmri.compute_event_avg(data, cond_onsets, n_time_points)
print("Condition average (cond_avg) shape:", cond_avg.shape)

In [ ]:
# Show a bunch of flatmaps in sequence
vkw = dict(mask=mask, cmap='RdBu_r', vmin=-2, vmax=2)
for n, ca in enumerate(cond_avg):
    try:
        plt.figure()
        _ = cx.quickflat.make_figure(cx.Volume(ca, sub, xfm, **vkw), height=300)
        plt.title('{n} TRs from onset'.format(n=n))
        plt.show()
    except:
        pass

## Discussion
How does the event average change over time? Why do you think this is? 

## Breakout session
> Do this for all conditions; make your code compact, if you can! Keep your results in a dictionary called event_avg

In [ ]:
### STUDENT ANSWER
event_avg = {}
event_avg_vol = {}
conditions = ['body', 'face','object','place','scramble']
n_time_points = 10


# Accounting for the HRF

we computed event-related averages for responses to different categories of objects. We saw that response in fMRI emerge slowly (2-3 TRs or 4-6 seconds) after the onset of a stimulus. Fundamentally, the most common thing that we want to know in fMRI experiments is how some event is related to brain responses. The event related averages show us that some areas of the brain respond more to some stimuli than others, but we would like a more firm statistical basis before we draw any conclusions from our data.

Let's account for the delay in the fMRI signal after a stimulus or other event.

### fMRI responses do not occur immediately after a stimulus

fMRI responses emerge slowly after the onset of a stimulus (or any other event). This means that, if we want to make an accurate mathematical model of how the brain responds, we have to somehow incorporate this delay into our model. 

To do this, we will borrow a concept from signal processing theory called the impulse response function. An impulse response function describes the way a signal emerges (for any system) after an event. The way the BOLD response emerges after an experimental event is called the *hemodynamic response function* or HRF. 

A great deal of early fMRI research went into accurately describing how the BOLD signal rises, falls, and resets to baseline after an event. We will rely on the conclusions of this research without going into much detail about it. For an overview of hemodynamic responses in fMRI, check out [this blog post](https://andysbrainbook.readthedocs.io/en/latest/fMRI_Short_Course/Statistics/03_Stats_HRF_Overview.html), and the papers linked in the lecture slides (Handwerker, Bandettini et al 2012; Logothetis & Wandell, 2004). The practical upshot of this work is that BOLD responses have a fairly characteristic shape, which is well described by a mathematical function called a *gamma function*. 

We have provided you with a function to produce this canonical HRF within the neurods module: 

    neurods.fmri.hrf()

In [ ]:
# Set the TR, or repetition time, a.k.a. the sampling rate for our data

TR = 1.0 # One measurement per second

# let's import a function that makes hrfs:
from neurods.fmri import hrf as generate_hrf

# Get a canonical hrf_1
t_hrf, hrf_1 = generate_hrf(tr=TR)
print('hrf_1 shape is', hrf_1.shape)
__ = plt.plot(t_hrf,hrf_1)


In [ ]:
# This is a response to a single discrete stimulus appearing at time 0, like this:
t = t_hrf
stimulus = np.zeros(t.shape)
stimulus[0] = 1

# We will be plotting stimulus / response pairs several times, so let's 
# just make a function for this right away.
def stim_resp_plot(t, stimulus, response, yl=(-0.2, 1.2)):
    """Plot stimulus and response"""
    plt.figure(figsize=(10,4))
    # Note stem() function!
    plt.stem(t, stimulus, linefmt='k-', markerfmt='.', basefmt='k-', label='Stimulus')
    plt.plot(t, response, 'r.-', label='BOLD response')
    plt.ylim(yl)
    plt.xlim([-1,t.max()+1])
    plt.xlabel('Time (seconds)')
    plt.ylabel('Response (arbitrary units)')
    _ = plt.legend()

# Plot
stim_resp_plot(t, stimulus, hrf_1)

---

Play with the parameters of the generate_hrf() function. See what happens if you change them. 

In [ ]:
### STUDENT ANSWER

Every time an event occurs, this slow hemodynamic response emerges. We will slowly explore how this hemodynamic response can affect the signal in a voxel.

First, we start with a hypothetical run of 200 TRs, in which no stimuli is presented.

In [ ]:
n = 200 # Total time points (TRs)
t = np.arange(n,)
# no stimulus
stimuli = np.zeros((n))
# we assume no response
response = np.zeros((n))
# here we plot the function
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))
# and see nothing as predicted.

We saw above how the time changes when we have a stimulus at time 0. Imagine we have a stimulus at time i=10. What would happen then to the activity? Just assume that this stimulus will create an hrf that will be *added* to the signal from times i to times i + hrf_length (hrf_length is the length of our theoretical hrf, which was 32 above).

Attention: make sure you modify response by adding something to it's values, and not just changing it.

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

i = 10
stimuli[i] = 1

# now add the response to stimulus i to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER


Now let's say that there were 3 stimuli, one at 10, one at 70 and one at 150, plot the resulting activity. 

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [10, 70, 150]
for i in stim_times:
    stimuli[i] = 1

# now add the response to those stimuli i, then plot using stim_resp_plot like above
### STUDENT ANSWER

Now say that the stimuli are closer together than the length of the hemodynamic function: let's say they occur at times 10, 21, 25, 70, 71,74, 75, 80 and 150, what happens? 

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [10,21,25, 70, 75, 80, 150]
for i in stim_times:
    stimuli[i] = 1

# now add the response to stimulus i to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER


What happens to your script if the stimulus appears at time 180? If it breaks, you need to make a change involving the min function.

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
response = np.zeros((n))

stim_times = [190]
for i in stim_times:
    stimuli[i] = 1

# now add the response to stimulus i to the response, then plot using stim_resp_plot like above
### STUDENT ANSWER


Let's write a function that will produce the correct response vectors given the stimulus vector and the hrf_1 vector. The function will go over every element in the stimulus vector. 

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))

stim_times = [10,21,25, 70, 75, 80, 150]
for i in stim_times:
    stimuli[i] = 1

# let's define our function here, then use it to generate output and plot the above plots:
def gen_responses(stimulus_vec, hrf_canonical):
    n = stimulus_vec.shape[0]
    hrf_length = len(hrf_canonical)
    response = np.zeros((n,))
    for i in range(n):
        if stimulus_vec[i]==1: # one way to do it: only if I have a stimulus, I will add a response
            index = range(i, min(i+hrf_length,n) )
            response[index] += hrf_canonical[:len(index)]
    return response

response = gen_responses(stimuli,hrf_1)
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))


Now assume that the stimulus occurs with different intensities: for example, a tactile stimulus with different levels of intensities. The assumption is that the response from the occurence of each feature is proportional to the intensity of the feature.

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = len(hrf_1)
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))

stim_times = [10,21,25, 70, 75, 80, 150]
stim_intensities = [0.25,0.25,1, 1, 1, 0.5, 0.5]
for idx, i_time in enumerate(stim_times):
    stimuli[i_time] = stim_intensities[idx]

def gen_responses(stimulus_vec, hrf_canonical):
    n = stimulus_vec.shape[0]
    hrf_length = len(hrf_canonical)
    response = np.zeros((n,))
    for i in range(n):
        index = range(i, min(i+hrf_length,n) )
        response[index] +=  stimuli[i]*hrf_canonical[:len(index)]
    return response

response = gen_responses(stimuli,hrf_1)
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

Now let's say we have a blocked stimulus, i.e. the simulus is on for 30 seconds, starting at 10, 70 and 130:



In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))

stim_times = list(range(10,40)) + list(range(70,100))+ list(range(130,160))
for i in stim_times:
    stimuli[i] = 1

# we can use your function here again, then use it to generate output and plot the above plots:
def gen_responses(stimulus_vec, hrf_canonical):
    n = stimulus_vec.shape[0]
    hrf_length = len(hrf_canonical)
    response = np.zeros((n,))
    for i in range(n):
        index = range(i, min(i+hrf_length,n) )
        response[index] +=  stimuli[i]*hrf_canonical[:len(index)]
    return response

response = gen_responses(stimuli,hrf_1)
stim_resp_plot(t, stimuli, response, yl=(-0.2, 1.2))

Effectively, a train of 30 spikes at the sampling frequency 1s is the same as having a constant step function for 30 seconds. What we did above is called a convolution: we have a signal, the stimulus. This signal is going to be modified by a function, the hrf in our case. Convolution is an integration operation where you integrate an initial signal with a modifying function, in a way that reflects how previous values of the signal (before time i) affect the new transformed signal (at time i). What we did above is a simplified integration: we are working in discrete time and we manually added the contributions at each time. 

This is what we did:
Let's call $T$ the length of the HRF. We went though the stimulus and made it affect the response at later stages. At the end, every value of the response $r(i)$ will have contributions to the stimulus $s$ from the previous $T$ time points:

$r(i) = s(i-1) \times hrf(1) +  s(i-2) \times hrf(2) + ... +  s(i-T+1)  \times hrf(T) \\
  \ \ \   = \Sigma_{\tau=1}^{T} s(i-\tau) \times h(\tau)$

This is exactly the expression of the discrete convolution function between functions $s$ and $h$. In continuous time, the convolution operation is written as:

$r(i) = \int_{\tau=1}^{T} s(i-\tau) \times h(\tau) = s \ast h(i)$

Here we will use a numpy function that is already implemented for this:

In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
#response = np.zeros((n))

stim_times = list(range(10,40)) + list(range(70,100))+ list(range(130,160))
for i in stim_times:
    stimuli[i] = 1

response = np.convolve(stimuli, hrf_1, mode='full')
print('resulting response has length {} and should be cropped'.format(len(response)))
# Here we also have to crop the signal because np.convolve creates a signal longer than n
# because it computes the response of stimuli appearing up to time n-1, which affect the 
# signal for longer than n.
stim_resp_plot(t, stimuli, response[:n], yl=(-0.2, 1.2))

Let's look at another example: the convolution of a simple periodic signal with the HRF function.


In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
#response = np.zeros((n))

stim_times = range(0,200,20)
for i in stim_times:
    stimuli[i] = 1
    
response = np.convolve(stimuli, hrf_1, mode='full')
# Here we also have to crop the signal!
stim_resp_plot(t, stimuli, response[:n], yl=(-0.2, 1.2))


And now of overlapping events:



In [ ]:
t_hrf, hrf_1 = generate_hrf(tr=TR)
hrf_length = 32
n = 200
t = np.arange(n,)
stimuli = np.zeros((n))
#response = np.zeros((n))

idx = np.arange(0, n, 20)
stimuli[idx] = 1
add = np.arange(len(idx), 0, -1)
stimuli[idx+add] = 1 
    
response = np.convolve(stimuli, hrf_1, mode='full')
# Here we also have to crop the signal!
stim_resp_plot(t, stimuli, response[:n], yl=(-0.2, 1.2))


### Important Observation 
So far we have assumed that every stimulus occurence creates a response in the voxel equal to the canonical hrf (and can be scaled by the feature value).

However, voxels in different parts of the brain can be differently responsive to a stimulus, or not responsive at all. In fMRI, we are interested in finding how responsive different voxels are to a stimulus. We will therefore introduce a new parameter: $w^v$, that describes the strength with which a voxel $v$ responds to the stimulus:

$ r^v(i) = w^v \times  (s \ast hrf (i) )$

We already know how to compute $s \ast hrf (i)$. In the previous examples we implicitely used $w^v = 1$, and assumed the voxel responds to the stimulus. We will gradually learn how we can estimate $w^v$ from the data, i.e. try to find how responsive voxel $v$ is to a stimulus, if at all. 


Next, we will show you two ways to account for this delay when trying to assess the relationship between brain signals and stimulus events. 

## Let's go back to the data
For these exercises, we will work with the data from a subset of voxels for the faces condition. We use the same plotting function, but we plot the data that is provided to us instead of forming the responses ourselves.

In [ ]:
stimulus = design_run1[:, 1] #face condition
data_sim = np.mean(data[:, [6,57,37]], axis=1) #averaging over ffa voxels
t = np.arange(1, data_sim.shape[0]+1, 1)
stim_resp_plot(t, stimulus, data_sim, yl=(-2, 5))

In [ ]:
plt.figure()
plt.scatter(stimulus, data_sim);
plt.show()
print("the correlation between the stimulus and the data is {}".format(np.corrcoef(stimulus, data_sim)[0,1]))

We know that the presentation of the stimulus should create a hemodynamic response if this voxel is sensitive to that stimulus. We therefore need to convolve the stimulus first with the hemodynamic response. 


In [ ]:
t2, hrf_2 = nds.fmri.hrf(tr=1)
plt.figure()
plt.plot(t2, hrf_2);
plt.show()

Now we can convolve the stimulus:

In [ ]:
conv_stimulus = np.convolve(stimulus, hrf_2, mode='full')[:120]
stim_resp_plot(t, conv_stimulus, data_sim, yl=(-2, 5))


In [ ]:
plt.figure()
plt.scatter(conv_stimulus, data_sim);
plt.show()
print("the correlation between the stimulus and the data is {}".format(np.corrcoef(conv_stimulus, data_sim)[0,1]))

We can see now we are able to recover a clearer relationship between the stimulus and the data. What is the variance of the noise that we can guess from this plot? Usually in fMRI we are not so lucky to have effects that are very clear. We will study in future lectures how to can expand this analysis.

Can we estimate from this data the magnitude of the weight $w_v$?

In [ ]:
# this function first a straight line through the points above
slope, intaercept = np.polyfit(data_sim, conv_stimulus, 1)
print(slope)

Remember, what is units of this value? FMRI signal doesn't have a unit and can be rescaled and normalized. The weight therefore depends on how the data is normalized and is meaningful only with respect to the variance of the data. 
